In [1]:
import pandas as pd
import numpy as np
import random
import math

In [8]:
data = pd.read_csv("~/Data/clean_rating3.csv")

idx_to_user = list(set(data[data.rating > 0].user_id))
idx_to_item = list(set(data[data.rating > 0].anime_id))

user_to_idx = {user: idx for idx, user in enumerate(idx_to_user)}
item_to_idx = {item: idx for idx, item in enumerate(idx_to_item)}

ratings = data[data.rating > 0].rating.values.tolist()
data = data[data.rating > 0].loc[:,['user_id', 'anime_id']].values
print(data.shape)

F, alpha, lam_bda, batch_size = 100, 0.02, 0.01, 512000
num_epochs, n, k = 30, len(ratings), 1 / math.sqrt(F)

P = np.array([[random.random() * k for _ in range(F)] for _ in range(len(idx_to_user))])
Q = np.array([[random.random() * k for _ in range(len(idx_to_item))] for _ in range(F)])
index = [i for i in range(data.shape[0])]

for epoch in range(num_epochs):
    sample = random.sample(index, batch_size)
    loss = 0
    for i in range(batch_size):
        user = user_to_idx[data[sample[i], 0]]
        item = item_to_idx[data[sample[i], 1]]
        rui = ratings[sample[i]]
        
        eui = rui - P[user, :].dot(Q[:, item])
        loss += eui
        for f in range(F):
            P[user, f] += alpha * (eui * Q[f, item] - lam_bda * P[user, f])
            Q[f, item] += alpha * (eui * P[user, f] - lam_bda * Q[f, item])

    alpha *= 0.9
    print(epoch + 1, loss)

(2208837, 2)
1 378443.29580344487


KeyboardInterrupt: 